# **Material auxiliar - Minicurso 03 (SBSeg 2025 - PUCPR)**

**Aviso:** Este notebook é **apenas uma demonstração didática** de criptografia híbrida pós-quântica. **Este código não deve ser usado em operação real**. Trata-se de um fluxo simples para fins únicos de aprendizado e ilustração, realizado através da combinação do ML-KEM, HKDF, AES-GCM e ML-DSA.

**Bibliotecas criptográficas utilizadas:**

**(1) pqcrypto**: biblioteca de criptografia pós-quântica com bindings para algoritmos padronizados pelo NIST.

https://pypi.org/project/pqcrypto/

**(2) pycryptodome**: biblioteca de criptografia convencional em Python.

https://pypi.org/project/pycryptodome/

**Para outras iniciativas, consultar:**

**(a) liboqs:** biblioteca em C da Open Quantum Safe, com suporte a KEMs e assinaturas pós-quânticas.

https://github.com/open-quantum-safe/liboqs

In [1]:
!pip -q install pqcrypto pycryptodome


**Verificação de variáveis de sistema e dependências**

In [2]:
import sys, platform
print("Python:", sys.version.split()[0])
print("Plataforma:", platform.platform())

try:
    import pqcrypto
    from Crypto.Cipher import AES
    print("pqcrypto OK  | pycryptodome OK")
except Exception as e:
    print("Dependências faltando:", e)


Python: 3.13.5
Plataforma: Linux-6.14.0-28-generic-x86_64-with-glibc2.39
pqcrypto OK  | pycryptodome OK


**Importação de módulos/bibliotecas**

In [3]:
import time, hmac, hashlib
from secrets import compare_digest
from textwrap import fill

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import HKDF

from pqcrypto.kem   import ml_kem_512   as KEM  # ML-KEM-512
from pqcrypto.sign  import ml_dsa_44    as SIG  # ML-DSA-44

**Funções utilitárias**

In [4]:

# Uma função trivial para formatação do título nos prints de cada célula
def title(t):
    bar = "=" * len(t)
    print(f"\n{bar}\n{t}\n{bar}")

# Uma função trivial para mostrar o prefixo em hex
def hexdump(b: bytes, max_bytes: int = 24) -> str:
    if not isinstance(b, (bytes, bytearray)): return str(b)
    hx = b.hex()
    return hx[:2*max_bytes] + ("..." if len(b) > max_bytes else "")

# Uma função para executar uma função várias vezes e medir o tempo total gasto
def bench(fn, *args, repeat=7):
    t0 = time.perf_counter()
    out = None
    for _ in range(repeat):
        out = fn(*args)
    t1 = time.perf_counter()
    return out, (t1 - t0) / repeat

# Uma função simples para verificar igualdade
def check_equal(a: bytes, b: bytes, label=""):
    ok = compare_digest(a, b)
    print(f"[CHECK] {label} iguais? {ok}")
    if not ok:
        raise AssertionError(f"Falha de verificação: {label}")
    return ok


**Geração de chaves**

In [5]:
title("1) Geração de chaves PQC (ML-KEM-512 e ML-DSA-44)")

print("\n")

# KEM: geração do par (pk, sk) para troca de segredos
(pk_kem, sk_kem), t_kem = bench(KEM.generate_keypair)
print(f"[KEM] Gera par de chaves ML-KEM-512 em {t_kem:.6f} s")
print("pk_kem bytes:", len(pk_kem), "| sk_kem bytes:", len(sk_kem))
print("pk_kem[0:24]:", hexdump(pk_kem))

print("\n")

# ML-DSA: geração do par (pk, sk) para assinatura
(pk_sig, sk_sig), t_sig = bench(SIG.generate_keypair)
print(f"[SIG] Gera par de chaves ML-DSA-44 em {t_sig:.6f} s")
print("pk_sig bytes:", len(pk_sig), "| sk_sig bytes:", len(sk_sig))
print("pk_sig[0:24]:", hexdump(pk_sig))


1) Geração de chaves PQC (ML-KEM-512 e ML-DSA-44)


[KEM] Gera par de chaves ML-KEM-512 em 0.000244 s
pk_kem bytes: 800 | sk_kem bytes: 1632
pk_kem[0:24]: 730892b5262edc988b5451b869cb3f82f469df8818c97355...


[SIG] Gera par de chaves ML-DSA-44 em 0.000233 s
pk_sig bytes: 1312 | sk_sig bytes: 2560
pk_sig[0:24]: b6e0774f99e3ab70817bf1beaa21026ca961cf6beab56547...


**Estabelecimento do segredo**

In [6]:
title("2) Estabelecimento de segredo (Encapsulação/Decapsulação (ML-KEM))")

# Bob encapsula um segredo para a pk_kem de Alice
(ct, ss_bob), t_encap = bench(KEM.encrypt, pk_kem)
print("\n")
print(f"[Bob] Encapsula segredo | tempo médio: {t_encap:.6f} s")
print("ct bytes:", len(ct), "| ct[0:24]:", hexdump(ct))
print("\n")

# Alice decapsula o segredo com sua sk_kem
(ss_alice), t_decap = bench(KEM.decrypt, sk_kem, ct)
print(f"[Alice] Decapsula segredo com sk_kem | tempo médio: {t_decap:.6f} s")
print("\n")

# Comparação
check_equal(ss_bob, ss_alice, "Segredo compartilhado")
print("ss[0:24]:", hexdump(ss_alice))

shared_secret = ss_alice


2) Estabelecimento de segredo (Encapsulação/Decapsulação (ML-KEM))


[Bob] Encapsula segredo | tempo médio: 0.000385 s
ct bytes: 768 | ct[0:24]: 8c89d571609a4834b0e0a59ac707be3c7f9e9ec06a2e2177...


[Alice] Decapsula segredo com sk_kem | tempo médio: 0.000239 s


[CHECK] Segredo compartilhado iguais? True
ss[0:24]: 5a409944e1eb5079769c4e4da729b9ef737a344a3acad983...


**Derivação da chave simétrica**

In [7]:
title("3) Derivação de chave simétrica (HKDF-SHA256) a partir do segredo")

#https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Cr2.pdf
#https://link.springer.com/chapter/10.1007/978-3-642-14623-7_34

from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA256

# Parâmetros de derivação
salt = get_random_bytes(16)
info = b"demo-hibrido-pqc/aes-gcm/v1"

# HKDF: deriva 32 bytes para chave AES-256
# Assinatura: HKDF(master, key_len, salt, hashmod, num_keys=1, context=None)
aes_key = HKDF(
    master=shared_secret,   # segredo do KEM
    key_len=32,
    salt=salt,
    hashmod=SHA256,
    num_keys=1,
    context=info
)

print("\n")

print("HKDF salt[0:16]:", hexdump(salt, 16))
print("HKDF info:", info)
print("AES-256 key[0:24]:", hexdump(aes_key))
print("[CHECK] Tamanho da chave AES:", len(aes_key), "bytes (esperado: 32)")



3) Derivação de chave simétrica (HKDF-SHA256) a partir do segredo


HKDF salt[0:16]: 200dfb209cbe32c5b8aa2ea062483936
HKDF info: b'demo-hibrido-pqc/aes-gcm/v1'
AES-256 key[0:24]: afb595c158ba910b83d61c24721eb1ed6da64fcb63f94ec6...
[CHECK] Tamanho da chave AES: 32 bytes (esperado: 32)


**Encriptação e Desencriptação**

In [8]:
title("4) Cifra autenticada (AEAD) com AES-GCM")

def aead_encrypt(key: bytes, plaintext: bytes, aad: bytes):
    nonce = get_random_bytes(12)              # 96 bits recomendado p/ GCM
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(aad)                        # vincula AAD à autenticação
    ct, tag = cipher.encrypt_and_digest(plaintext)  # sem pad, é stream-like
    return nonce, ct, tag

def aead_decrypt(key: bytes, nonce: bytes, ct: bytes, tag: bytes, aad: bytes) -> bytes:
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(aad)
    return cipher.decrypt_and_verify(ct, tag)

# Mensagem e cabeçalho (AAD)
mensagem = b"Mensagem confidencial via ML-KEM + AES-GCM"
aad      = b"header: rota=beta; versao=1; tipo=demo"

# Encriptação
print("\n")
(nonce, ct, tag), t_enc = bench(aead_encrypt, aes_key, mensagem, aad)
print(f"[Alice] Realiza a encriptação com AES-GCM | tempo médio: {t_enc:.6f} s")
print("nonce bytes:", len(nonce), "| tag bytes:", len(tag))
print("aad (autenticada, nao cifrada):", aad)
print("ct bytes:", len(ct), "| ct[0:24]:", hexdump(ct))
print("\n")

# Desencriptação
(pt_rec), t_dec = bench(aead_decrypt, aes_key, nonce, ct, tag, aad)
print(f"[Bob] Realiza a desencriptação e verifica tag/AAD | tempo médio: {t_dec:.6f} s")
print("\n")

check_equal(pt_rec, mensagem, "Mensagem recebida")
print("Mensagem recuperada:", pt_rec)



4) Cifra autenticada (AEAD) com AES-GCM


[Alice] Realiza a encriptação com AES-GCM | tempo médio: 0.003127 s
nonce bytes: 12 | tag bytes: 16
aad (autenticada, nao cifrada): b'header: rota=beta; versao=1; tipo=demo'
ct bytes: 42 | ct[0:24]: 1e33bddd4f8914319e0e67d9ce766fe3338fdf60b485e15f...


[Bob] Realiza a desencriptação e verifica tag/AAD | tempo médio: 0.000166 s


[CHECK] Mensagem recebida iguais? True
Mensagem recuperada: b'Mensagem confidencial via ML-KEM + AES-GCM'


**Assinatura e verificação**

In [9]:
title("5) Assinatura digital PQC (ML-DSA)")

# Pacote a ser assinado (nonce || aad || ct || tag)
to_sign = nonce + aad + ct + tag
print("\n")
print("Dados a assinar (nonce|aad|ct|tag) bytes:", len(to_sign))
print("to_sign[0:24]:", hexdump(to_sign))

# Assinatura
print("\n")
(signature), t_sign = bench(SIG.sign, sk_sig, to_sign)
print(f"[Alice] Assina o pacote com ML-DSA-44 | tempo médio: {t_sign:.6f} s")
print("assinatura bytes:", len(signature), "| assinatura[0:24]:", hexdump(signature))

# Verificação
print("\n")
(ok), t_verify = bench(SIG.verify, pk_sig, to_sign, signature)
print(f"[Bob] Verificou assinatura | tempo médio: {t_verify:.6f} s")
print("Assinatura válida?", ok)
if not ok:
    raise AssertionError("Assinatura inválida.")



5) Assinatura digital PQC (ML-DSA)


Dados a assinar (nonce|aad|ct|tag) bytes: 108
to_sign[0:24]: 13ecec0be0890be3842668226865616465723a20726f7461...


[Alice] Assina o pacote com ML-DSA-44 | tempo médio: 0.000473 s
assinatura bytes: 2420 | assinatura[0:24]: 515bb68f139e474035d53f5f918109c639e3d435829b49c2...


[Bob] Verificou assinatura | tempo médio: 0.000588 s
Assinatura válida? True


**Verificação de Tamanhos e Tempos**

In [10]:
title("6) Tamanhos e Tempos (médias)")

print("\n")
print("TAMANHOS (bytes)")
print("- pk_kem (ML-KEM-512) :", len(pk_kem))
print("- sk_kem (ML-KEM-512) :", len(sk_kem))
print("- ct (KEM ciphertext) :", len(ct))
print("- ss (shared secret)  :", len(shared_secret))
print("- aes_key (HKDF-256)  :", len(aes_key))
print("- pk_sig (ML-DSA-44)  :", len(pk_sig))
print("- sk_sig (ML-DSA-44)  :", len(sk_sig))
print("- assinatura (ML-DSA) :", len(signature))

print("\n")
print("\nTEMPOS (segundos, médias)")
print("- Geração de chaves KEM :", f'{t_kem:.6f}')
print("- Geração de chaves SIG :", f'{t_sig:.6f}')
print("- Encapsulação (KEM)    :", f'{t_encap:.6f}')
print("- Decapsulação (KEM)    :", f'{t_decap:.6f}')
print("- AES-GCM encrypt       :", f'{t_enc:.6f}')
print("- AES-GCM decrypt+verify:", f'{t_dec:.6f}')
print("- Assinar (ML-DSA)      :", f'{t_sign:.6f}')
print("- Verificar (ML-DSA)    :", f'{t_verify:.6f}')



6) Tamanhos e Tempos (médias)


TAMANHOS (bytes)
- pk_kem (ML-KEM-512) : 800
- sk_kem (ML-KEM-512) : 1632
- ct (KEM ciphertext) : 42
- ss (shared secret)  : 32
- aes_key (HKDF-256)  : 32
- pk_sig (ML-DSA-44)  : 1312
- sk_sig (ML-DSA-44)  : 2560
- assinatura (ML-DSA) : 2420



TEMPOS (segundos, médias)
- Geração de chaves KEM : 0.000244
- Geração de chaves SIG : 0.000233
- Encapsulação (KEM)    : 0.000385
- Decapsulação (KEM)    : 0.000239
- AES-GCM encrypt       : 0.003127
- AES-GCM decrypt+verify: 0.000166
- Assinar (ML-DSA)      : 0.000473
- Verificar (ML-DSA)    : 0.000588


**Níveis do ML-KEM (512/768/1024) e ML-DSA (44/65/87)**

In [11]:
title("Comparação de níveis (ML-KEM (512/768/1024) e ML-DSA (44/65/87))")

from pqcrypto.kem  import ml_kem_768 as KEM768, ml_kem_1024 as KEM1024
from pqcrypto.sign import ml_dsa_65  as SIG65,   ml_dsa_87   as SIG87

def kem_roundtrip(KEM_MOD):
    (pk, sk), tk = bench(KEM_MOD.generate_keypair)
    (ct, ss1), te = bench(KEM_MOD.encrypt, pk)
    (ss2), td = bench(KEM_MOD.decrypt, sk, ct)
    check_equal(ss1, ss2, f"ss ({KEM_MOD.__name__})")
    return {
        "pk": len(pk), "sk": len(sk), "ct": len(ct),
        "t_gen": tk, "t_enc": te, "t_dec": td
    }

def sig_roundtrip(SIG_MOD, payload=b"benchmark"):
    (pk, sk), tk = bench(SIG_MOD.generate_keypair)
    (sig), ts = bench(SIG_MOD.sign, sk, payload)
    (ok), tv = bench(SIG_MOD.verify, pk, payload, sig)
    return {
        "pk": len(pk), "sk": len(sk), "sig": len(sig),
        "t_gen": tk, "t_sign": ts, "t_verify": tv, "ok": ok
    }

print("\n[KEM] ML-KEM-512")
r512 = kem_roundtrip(KEM)
print(r512)

print("\n[KEM] ML-KEM-768")
r768 = kem_roundtrip(KEM768)
print(r768)

print("\n[KEM] ML-KEM-1024")
r1024 = kem_roundtrip(KEM1024)
print(r1024)

print("\n[SIG] ML-DSA-44")
s44 = sig_roundtrip(SIG)
print(s44)

print("\n[SIG] ML-DSA-65")
s65 = sig_roundtrip(SIG65)
print(s65)

print("\n[SIG] ML-DSA-87")
s87 = sig_roundtrip(SIG87)
print(s87)



Comparação de níveis (ML-KEM (512/768/1024) e ML-DSA (44/65/87))

[KEM] ML-KEM-512
[CHECK] ss (pqcrypto.kem.ml_kem_512) iguais? True
{'pk': 800, 'sk': 1632, 'ct': 768, 't_gen': 7.745314360363409e-05, 't_enc': 7.938714302976482e-05, 't_dec': 9.698128581346412e-05}

[KEM] ML-KEM-768
[CHECK] ss (pqcrypto.kem.ml_kem_768) iguais? True
{'pk': 1184, 'sk': 2400, 'ct': 1088, 't_gen': 0.00010916728621980707, 't_enc': 0.00011756857150300805, 't_dec': 0.00013577828628643016}

[KEM] ML-KEM-1024
[CHECK] ss (pqcrypto.kem.ml_kem_1024) iguais? True
{'pk': 1568, 'sk': 3168, 'ct': 1568, 't_gen': 0.00016206814325414598, 't_enc': 0.00014015299945770363, 't_dec': 0.00011897628532356716}

[SIG] ML-DSA-44
{'pk': 1312, 'sk': 2560, 'sig': 2420, 't_gen': 8.905214261696009e-05, 't_sign': 0.0004026418568433395, 't_verify': 0.00011562871389157538, 'ok': True}

[SIG] ML-DSA-65
{'pk': 1952, 'sk': 4032, 'sig': 3309, 't_gen': 0.0002006107138835692, 't_sign': 0.0004296357136419309, 't_verify': 0.0001365228578963849, 'o